# Analisi statistica dei dati relativi al decadimento della particella K*

## 1. Introduzione
La particella K* è altamente instabile, e ha una vita media di 10<sup>-11</sup> secondi. Per questo motivo, se ne si vuole provarne l'esistenza si è costretti a ricorrere a metodi statistici, piuttosto che a a metodi di rilevamento diretto; questo è lo scopo della presente esperienza di laboratorio. L'analisi statistica che è stata svolta è basata su dati ottenuti tramite una simulazione. In particolare, il codice che è stato scritto scritto si occupa di:

1. Simulare degli eventi di collisione dell'esperimento _ALICE_ tramite il metodo di generazione Montecarlo.
2. Analizzare i dati raccolti, dimostrare l'esistenza della particella K* e verificare i risultati ottenuti con un istogramma di controllo.

## 2. Struttura del codice
Il codice di simulazione è stato realizzato in _C++_ e fa un uso intensivo delle librerie _ROOT_. È stato scelto un approccio diverso da quello consigliato, in modo da espandere ancora di più le possibilità di reimpiego del codice. Segue una breve descrizione di quello che è stato sviluppato.

### 2.1. Classi implementate
Sono state implementate due classi virtuali: `Entity` ed `Experiment`.

- #### Entity
  Una _Entity_ rappresenta una generica entità rilevabile da un esperimento (i.e. particella o risonanza), a cui sono associate una quantità di moto, una carica e una massa. Una _Entity_ può essere solo di un numero finito di tipi, ognuno definito da tre proprietà: carica, massa e larghezza di risonanza (quest'ultima esiste solo per le risonanze). Ogni tipo è implementato tramite una classe derivata di `Entity`, in cui sono dichiarate le rispettive proprietà; per risparmiare memoria, sono dichiarate come `static`.  
  I metodi e le proprietà condivise tra tutti i tipi, invece, sono salvati nella classe base `Entity`.
  
- #### Experiment
  La classe `Experiment` rappresenta un esperimento dell'_LHC_. Il codice che si occupa di simulare gli eventi di collisione e raccogliere i dati risiede qua. Per dare la possibilità di riutilizzare questo programma per esperimenti diversi, ogni esperimento è implementato tramite una classe derivata di `Experiment`. L'esperimento che interessa a noi è implementato in `KaonSDecay` ed è descritto accuratamente in <ins>[3. Generazione di dati]</ins>.

### 2.2. Meccanismi di reimpiego
Il polimorfismo dinamico ha reso possibile sviluppare il codice con una struttura modulare. Per aggiungere nuovi tipi di particelle o esperimenti, basta semplicemente aggiungere un file in cui se ne definiscono le proprietà. Tutto questo è stato realizzato prestando particolare attenzione all'utilizzo delle risorse di calcolo: non sono presenti né dati duplicati né linee di codice ridondanti.

Il codice usato per la simulazione dei dati è disponibile interamente su
GitHub\[1\], insieme ad ulteriori informazioni e istruzioni per la
compilazione.

\[1\]: https://github.com/P2-718na/ALICE-simulation

## 3. Generazione dei dati

### 3.1. Funzionamento del programma
Il numero di eventi di un esperimento di _ALICE_ è dell'ordine di 10<sup>5</sup>-10<sup>7</sup>, ognuno dei quali conta 10<sup>2</sup>-10<sup>4</sup> particelle. Il programma simula gli eventi uno ad uno, svolgendo le seguenti operazioni:

1. Viene generato un certo numero di particelle <ins>[3.2. Generazione delle particelle]</ins>.
2. Vengono fatte decadere le particelle instabili <ins>[3.3. Decadimenti]</ins>.
3. Vengono inseriti tutti i dati raccolti in un istogramma di _ROOT_.
4. Viene ripulita la memoria prima dell'evento successivo.

### 3.2. Generazione delle particelle
In ogni evento, è presente solo un numero limitato di tipi di particelle, tutte indistingubili tra loro. Per la loro generazione, quindi, si è tenuto conto solamente di:

- _PDF_ della distribuzione dei momenti delle particelle, come descritto in tabella <ins>[A.1]</ins>.
- Frequenza relativa con cui appare ogni tipo di particella, come descritto in tabella <ins>[A.2]</ins>.

Tutte le particelle generate vengono salvate in memoria fino alla simulazione dell'evento successivo.
Per generare numeri secondo distribuzioni di probabilità, sono stati usati i metodi della classe `TRandom3` di _ROOT_.

### 3.3. Decadimenti
Non appena è generata una particella K*, viene rimossa e sostituita da una coppia $\pi$/$\kappa$ di segno opposto, per simulare il processo di decadimento. I prodotti del decadimento hanno la stessa energia della particella madre, divisa in modo casuale tra di loro.

Per consentire la creazione di un istogramma di controllo, questi vengono marchiati, così da poterli distinguere dalle particelle generate normalmente. In un esperimento reale non sarebbe possibile distinguere tra particelle prodotte dal decadimento e particelle generate da una collisione.

## 4. Analisi dei dati

### 4.1. Correttezza dei dati generati
- #### Abbondanza delle particelle
  Il numero di particelle generato corrisponde con quanto inserito nei dati di input e la percentuale di occorrenze di ogni singola particella è compatibile con quanto specificato nel programma, entro le incertezze. I dati sono riassunti in <ins>[B.1]</ins> e <ins>[A.3]</ins>
  
- #### Distribuzione angoli azimutali e polari
  Le distribuzioni attese sono uniformi. I risultati del fit, visibili in <ins>[B.2]</ins>, <ins>[B.3]</ins> e <ins>[A.4]</ins>, sono concordi con i valori attesi. Entrambi gli istogrammi sono divisi in $360$ bin e contengono $10^7$ elementi; il valore atteso per il parametro `p0` è: $10^7 / 360 = 2.778*10^4$, concorde con quanto ottenuto. Il fit è stato eseguito usando la funzione `pol0` di ROOT.
  
- #### Distribuzione modulo dell'impulso
  La distribuzione attesa è esponenziale, con media uno. I risultati del fit, visibili in <ins>[B.4]</ins> <ins>[A.4]</ins>, sono concordi con i valori attesi. Il fit è stato eseguito usando la funzione `expo` di ROOT.

### 4.2. Analisi degli istogrammi di massa invariante

Dal decadimento della K* si generano sempre coppie Pione-Kaone di segno opposto (come conseguenza della legge di conservazione della carica) e la cui massa invariante corrisponde proprio alla massa della K*. Questa osservazione permette di utilizzare il seguente approccio per estrarre il segnale della K*:

1. Si calcolano le masse invarianti di tutte le possibili coppie Pione/Kaone di *segno opposto*.
2. Si calcolano le masse invarianti di tutte le possibili coppie Pione/Kaone di *segno concorde*.
3. Si raccolgono i dati in due istogrammi (verranno chiamati rispettivamente istogramma "1" e "2"). 
4. Si sottrae (2) da (1)

Sia in (1) che in (2) si trovano dei picchi in corrispondenza di tutte le combinazioni di massa invariante delle particelle. In (1), però, visto che è presente una piccola quantità di coppie Pione/Kaone generate da decadimenti della K*, ci aspettiamo di trovare un altro picco proprio in corrispondenza della massa della K*. Il (2) permette di rimuovere il rumore di fondo da (1), evidenziando questo picco. In questo modo, ci si aspetta di ottenere un grafico risultante che segue la distribuzione della massa della K*, con deviazione standard corrispondente all'ampiezza di risonanza della particella.

In natura questa distribuzione è una Lorenziana; tuttavia, per semplicità, nel programma di generazione è stata usata una distribuzione Gaussiana. Le funzioni di fit sono state scelte di conseguenza.

L'istogramma ottenuto sottraendo (2) da (1) è consistente con quello ottenuto sottraendo (4) da (3).

### 3. Analisi della massa della K*
Ora possiamo analizzare i grafici ottenuti. Ci aspettiamo che i grafici appena ottenuti seguano una distribuzione gaussiana, con media corrispondente alla massa di K* e sigma corrispondente alla vita media di K*.

Nota: se questi fossero dati reali, ci aspetteremmo una distribuzione _lorentziana_. Otteniamo una distribuzione gaussiana perchè durante la generazione dei dati, abbiamo usato un algoritmo semplificato per simulare il decadimento di K*, ottenendo questo comportamento.

In [ ]:
//cv->cd();
//
//InvMassPKSubtract->Fit("gaus", "QB", "", 0, 2);
//InvMassPKSubtract->Draw("hist");
//cv->Draw();

La distribuzione gaussiana fitta perfettamente il grafico e i dati ottenuti sono coerenti con quelli attesi (entro le incertezze).

/*
hists_[TraverseMomentumDist] = std::make_unique<TH1F>("TraverseMomentumDist", "Traverse momentum", 500, 0, 4);
    hists_[EnergyDist]           = std::make_unique<TH1F>("EnergyDist", "Energy", 1000, 0, 6);
    hists_[InvMass]              = std::make_unique<TH1F>("InvMass", "Invariant mass", 500, 0, 8);
*/

## Appendice A: Tabelle

<table>
    <caption><b>Tabella A.1.</b> Parametri di generazione.</caption>
    <tr>
        <th>Parametro</th>
        <th>Valore</th>
    </tr>
    <tr>
        <td>n° eventi</td>
        <td>10<sup>5</sup></td>
    </tr>
    <tr>
        <td>n° particelle per evento</td>
        <td>100</td>
    </tr>
    <tr>
        <td><i>PDF</i> modulo del momento</td>
        <td>Esponenziale, media 1</td>
    </tr>
    <tr>
        <td><i>PDF</i> angoli azimutali e polari</td>
        <td>Uniforme</td>
    </tr>
</table>
<div style="page-break-after: always;"></div>

<table>
    <caption><b>Tabella A.2.</b> Probabilità di generazione delle particelle, configurazione inserita nel programma.</caption>
    <tr>
        <th>Particella</th>
        <th>Probabilità relativa</th>
    </tr>
    <tr>
        <td>$\pi$+</td>
        <td>40.0%</td>
    </tr>
    <tr>
        <td>$\pi$-</td>
        <td>40.0%</td>
    </tr>
    <tr>
        <td>$\kappa$+</td>
        <td>5.0%</td>
    </tr>
    <tr>
        <td>$\kappa$-</td>
        <td>5.0%</td>
    </tr>
    <tr>
        <td>p+</td>
        <td>4.5%</td>
    </tr>
    <tr>
        <td>p-</td>
        <td>4.5%</td>
    </tr>
    <tr>
        <td>$\kappa$*</td>
        <td>1.0%</td>
    </tr>
</table>
<div style="page-break-after: always;"></div>

<table>
    <caption><b>Tabella A.3.</b> Abbondanza delle particelle generate, cofrontata con il valore atteso. Le incertezze sui valori osservati sono valutate usando la regola della radice.</caption>
    <tr>
        <th>Particella</th>
        <th>Numero atteso ($10^5$ occorrenze)</th>
        <th>Numero ottenuto ($10^3$ occorrenze) </th>
        <th>Percentuale attesa</th>
        <th>Percentuale ottenuta</th>
    </tr>
    <tr>
        <td>$\pi$+</td>
        <td>40</td>
        <td>3999 $\pm$ 2</td>
        <td>40.0%</td>
        <td>40.0%</td>
    </tr>
    <tr>
        <td>$\pi$-</td>
        <td>40</td>
        <td>4000 $\pm$ 2</td>
        <td>40.0%</td>
        <td>40.0%</td>
    </tr>
    <tr>
        <td>$\kappa$+</td>
        <td>5</td>
        <td>500.3 $\pm$ 0.7</td>
        <td>5.0%</td>
        <td>5.0%</td>
    </tr>
    <tr>
        <td>$\kappa$-</td>
        <td>5</td>
        <td>500.1 $\pm$ 0.7</td>
        <td>5.0%</td>
        <td>5.0%</td>
    </tr>
    <tr>
        <td>p+</td>
        <td>4.5</td>
        <td>449.0 $\pm$ 0.7</td>
        <td>4.5%</td>
        <td>4.5%</td>
    </tr>
    <tr>
        <td>p-</td>
        <td>4.5</td>
        <td>450.7 $\pm$ 0.7</td>
        <td>4.5%</td>
        <td>4.5%</td>
    </tr>
    <tr>
        <td>$\kappa$*</td>
        <td>1</td>
        <td>100.5 $\pm$ 0.3</td>
        <td>1.0%</td>
        <td>1.0%</td>
    </tr>
    <tr>
        <td><b>Totale</b></td>
        <td>100</td>
        <td>10000 $\pm$ 7</td>
        <td>100.0%</td>
        <td>100.0%</td>
    </tr>
</table>
<div style="page-break-after: always;"></div>

<table>
    <caption><b>Tabella A.4.</b> Distribuzioni ottenute di angoli polari, azimutali e modulo dell'impulso.</caption>
    <tr>
        <th>Distribuzione</th>
        <th>Nome parametro del fit</th>
        <th>Valore parametro</th>
        <th>${\chi}^2$</th>
        <th>DOF</th>
        <th>${\chi}^2$/ DOF</th>
    </tr>
    <tr>
        <td>Angolo azimutale $\phi$ (<code>pol0</code>)</td>
        <td>p0 (intercetta)</td>
        <td>27780 $\pm$ 90</td>
        <td>357.6</td>
        <td>359</td>
        <td>0.996</td>
    </tr>
    <tr>
        <td>Angolo polare $\theta$ (<code>pol0</code>)</td>
        <td>p0 (intercetta)</td>
        <td>27780 $\pm$ 90</td>
        <td>337.2</td>
        <td>359</td>
        <td>0.939</td>
    </tr>
    <tr>
        <td>Modulo dell'impulso (<code>expo</code>)</td>
        <td>Slope (media esponenziale)</td>
        <td>1.0004 $\pm$ 0.0003</td>
        <td>527.2</td>
        <td>498</td>
        <td>1.06</td>
    </tr>
</table>
<div style="page-break-after: always;"></div>

# Appendice B: Grafici

<img src="../graphs/4_1.svg"/>

lkjn

# Appendice C: Codice